## The simpliest usage example of py_boost

### Installation (if needed)

In [1]:
# !pip install py-boost

### Imports

In [2]:
import os
# Optional: set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import joblib
from sklearn.datasets import make_regression

# simple case - just one class is used
from py_boost import GradientBoosting 

### Generate dummy regression data

In [3]:
%%time
X, y = make_regression(150000, 100, n_targets=10, random_state=42)
X_test, y_test = X[:50000], y[:50000]
X, y = X[-50000:], y[-50000:]

CPU times: user 2.07 s, sys: 1.47 s, total: 3.54 s
Wall time: 809 ms


### Train GBM

The only required argument here is the loss funnction. Type of solved task is defined by loss function and input target shape. It could be passed as Loss instance or via string alias:

* ***'mse'*** for the regression/multitask regression
* ***'msle'*** for the regression/multitask regression
* ***'bce'*** for the binary/multilabel classification
* ***'crossentropy'*** for the multiclassification


Training is simply done by calling .fit metod. Possible arguments:

* ***'X'*** 
* ***'y'*** 
* ***'sample_weight'*** 
* ***'eval_sets'*** Validation set is passed as the list of dict with possible keys ['X', 'y', 'sample_weight']. Note: if multiple valid sets are passed, the best model is selected using the last one

#### The example below illustrates how to train simple regression task:

In [4]:
%%time
model = GradientBoosting('mse')

model.fit(X, y[:, 0], eval_sets=[{'X': X_test, 'y': y_test[:, 0]},])

[03:05:41] Stdout logging level is INFO.
[03:05:41] GDBT train starts. Max iter 100, early stopping rounds 100
[03:05:41] Iter 0; Sample 0, rmse = 173.6750220057655; 
[03:05:41] Iter 10; Sample 0, rmse = 133.1954905849117; 
[03:05:41] Iter 20; Sample 0, rmse = 107.86632150004753; 
[03:05:42] Iter 30; Sample 0, rmse = 90.08244855794447; 
[03:05:42] Iter 40; Sample 0, rmse = 76.44527769256084; 
[03:05:42] Iter 50; Sample 0, rmse = 65.61042654240376; 
[03:05:42] Iter 60; Sample 0, rmse = 56.80163725665563; 
[03:05:42] Iter 70; Sample 0, rmse = 49.57703267292228; 
[03:05:42] Iter 80; Sample 0, rmse = 43.603722017581426; 
[03:05:42] Iter 90; Sample 0, rmse = 38.69769765751429; 
[03:05:42] Iter 99; Sample 0, rmse = 34.991929680312666; 
CPU times: user 4.03 s, sys: 664 ms, total: 4.69 s
Wall time: 3.45 s


### Train for multiregression

Each of built-in loss functions has its own default metric, so metric definition is optional. If you need to specify eval metric, you could pass Metric instance or use string alias:

#### Default metrics:

* ***'rmse'*** is the default for the ***'mse'*** loss
* ***'rmsle'*** is the default for the  ***'msle'*** loss
* ***'bce'*** is the default for the ***'bce'*** loss
* ***'crossentropy'*** is the default for the ***'crossentropy'*** loss

#### Non default metrics:

* ***'r2'*** for the regression/multitask regression
* ***'auc'*** for the binary classification
* ***'accuracy'*** for any classification task
* ***'precision'*** for any classification task
* ***'recall'*** for any classification task
* ***'f1'*** for any classification task


Also you can specify other common GBDT hyperparameters as shown below

#### The example below demonstrates the example of training model for multioutput regression task, no extra definition needed to switch the task, just pass multidimensional target:

In [5]:
%%time
model = GradientBoosting('mse', 'r2_score',
                         ntrees=1000, lr=.01, verbose=100, es=200, lambda_l2=1,
                         subsample=.8, colsample=.8, min_data_in_leaf=10, min_gain_to_split=0, 
                         max_bin=256, max_depth=6)

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[03:05:43] Stdout logging level is INFO.
[03:05:43] GDBT train starts. Max iter 1000, early stopping rounds 200
[03:05:43] Iter 0; Sample 0, R2_score = 0.008412879588052391; 
[03:05:44] Iter 100; Sample 0, R2_score = 0.5170943771682728; 
[03:05:46] Iter 200; Sample 0, R2_score = 0.7246209942553812; 
[03:05:48] Iter 300; Sample 0, R2_score = 0.8328702725409002; 
[03:05:49] Iter 400; Sample 0, R2_score = 0.8950261453129482; 
[03:05:51] Iter 500; Sample 0, R2_score = 0.9320781366121136; 
[03:05:53] Iter 600; Sample 0, R2_score = 0.9546805731420273; 
[03:05:54] Iter 700; Sample 0, R2_score = 0.9687201388800404; 
[03:05:56] Iter 800; Sample 0, R2_score = 0.9775984497641979; 
[03:05:58] Iter 900; Sample 0, R2_score = 0.9832880678482553; 
[03:05:59] Iter 999; Sample 0, R2_score = 0.9869727876125817; 
CPU times: user 16.9 s, sys: 1.7 s, total: 18.6 s
Wall time: 17.3 s


## Inference

#### Prediction could be done via calling .predict method

In [6]:
%%time
preds = model.predict(X_test)

preds.shape

CPU times: user 831 ms, sys: 476 ms, total: 1.31 s
Wall time: 1.31 s


(50000, 10)

In [7]:
preds

array([[-228.35983  , -138.24062  , -270.85815  , ..., -132.49632  ,
        -207.68971  , -227.21599  ],
       [-107.79079  , -106.12598  ,  -54.361683 , ..., -122.162834 ,
        -112.88282  ,  -15.996089 ],
       [ -38.86242  ,  -60.58174  ,  149.93024  , ...,   19.207836 ,
         -26.376608 , -211.70198  ],
       ...,
       [ -93.89007  ,  123.4826   ,   79.73774  , ...,  216.58656  ,
          29.11607  ,    6.3970094],
       [  -9.551284 ,  131.65627  ,  242.57858  , ...,  141.3406   ,
         166.46692  ,  204.59633  ],
       [ -28.870468 ,   27.83899  ,  167.77754  , ...,   89.18847  ,
          18.723934 ,   -5.1239767]], dtype=float32)

#### Prediction for certan iterations could be done via calling .predict_staged method

In [8]:
%%time
preds = model.predict_staged(X_test, iterations=[100, 300, 500])

preds.shape

CPU times: user 246 ms, sys: 225 ms, total: 471 ms
Wall time: 470 ms


(3, 50000, 10)

#### Tree leaves indicies prediction for certan iterations could be done via calling .predict_leaves method

In [9]:
%%time
preds = model.predict_leaves(X_test, iterations=[100, 300, 500])

preds.shape

CPU times: user 15.1 ms, sys: 3.84 ms, total: 18.9 ms
Wall time: 17.8 ms


(3, 50000, 1)

In [10]:
preds.T[0]

array([[ 4,  6, 32],
       [28,  6, 46],
       [ 1,  4, 21],
       ...,
       [29,  6, 32],
       [ 3, 29, 32],
       [19,  7, 32]], dtype=uint32)

#### Feature importances

In [11]:
model.get_feature_importance()

array([  39.,   51.,   38.,   35.,   67.,   42., 5539.,   42.,   52.,
         59.,   40.,   46.,   27.,   47.,   36., 5806., 5599.,   42.,
         36., 5492.,   24.,   39.,   47.,   47.,   27.,   33.,   49.,
         49.,   34.,   47.,   34.,   65.,   52.,   52.,   35.,   48.,
       5870.,   44.,   45.,   36.,   67.,   52.,   31.,   43.,   51.,
         38.,   41.,   52.,   57.,   47.,   65.,   50., 5954.,   49.,
         46.,   69.,   44.,   32.,   42.,   43.,   42.,   48.,   41.,
         55.,   47.,   35.,   45.,   45.,   52.,   62.,   43.,   33.,
         37.,   46.,   43.,   47.,   62.,   46.,   43.,   60.,   53.,
         58.,   44.,   27.,   52.,   35., 5751., 3570.,   56., 5829.,
         47., 6173.,   48.,   35.,   54.,   47.,   37.,   42.,   46.,
         59.], dtype=float32)

#### Trained model could be saved as pickle for inference

In [12]:
joblib.dump(model, 'temp_model.pkl')

new_model = joblib.load('temp_model.pkl')
new_model.predict(X_test)

array([[-228.35983  , -138.24062  , -270.85815  , ..., -132.49632  ,
        -207.68971  , -227.21599  ],
       [-107.79079  , -106.12598  ,  -54.361683 , ..., -122.162834 ,
        -112.88282  ,  -15.996089 ],
       [ -38.86242  ,  -60.58174  ,  149.93024  , ...,   19.207836 ,
         -26.376608 , -211.70198  ],
       ...,
       [ -93.89007  ,  123.4826   ,   79.73774  , ...,  216.58656  ,
          29.11607  ,    6.3970094],
       [  -9.551284 ,  131.65627  ,  242.57858  , ...,  141.3406   ,
         166.46692  ,  204.59633  ],
       [ -28.870468 ,   27.83899  ,  167.77754  , ...,   89.18847  ,
          18.723934 ,   -5.1239767]], dtype=float32)